In [1]:
%load_ext autoreload
%autoreload 2

## Step 1: Load in Data

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import project_engie_scada
from project_engie_scada import Project_Engie
from operational_analysis.methods import turbine_long_term_gross_energy
from operational_analysis.methods import electrical_losses

In [3]:
engie_project = project_engie_scada.Project_Engie(path= '.')

In [4]:
engie_project.prepare()

INFO:project_engie_scada:Loading SCADA data
INFO:operational_analysis.types.timeseries_table:Loading name:engie_scada
INFO:project_engie_scada:SCADA data loaded
INFO:project_engie_scada:Timestamp QC and conversion to UTC
INFO:project_engie_scada:Correcting for out of range of power, wind speed, and wind direction variables
INFO:project_engie_scada:Flagging unresponsive sensors
INFO:project_engie_scada:Converting field names to IEC 61400-25 standard
INFO:operational_analysis.types.timeseries_table:Loading name:engie_meter
INFO:project_engie_scada:Loading Availability and Curtailment Data
INFO:operational_analysis.types.timeseries_table:Loading name:engie_avail_curt
INFO:operational_analysis.types.timeseries_table:Loading name:merra2_data
INFO:operational_analysis.types.timeseries_table:Loading name:ncep2_data
INFO:operational_analysis.types.timeseries_table:Loading name:erai_data


## Step 2: Inspect Scada for any irregularities

In [5]:
scada_df = engie_project._scada.df
scada_df.head()

,time,wtur_W_avg,wmet_HorWd_Dir,wmet_wdspd_avg,id,energy_kwh
time,,,,,,
2013-01-01 00:00:00,2013-01-01 00:00:00,1303770.0,174.13000,9.479999,R80711,217.295000
2013-01-01 00:10:00,2013-01-01 00:10:00,1276150.0,173.53000,9.240000,R80711,212.691667
2013-01-01 00:20:00,2013-01-01 00:20:00,1286340.0,172.52000,9.190000,R80711,214.390000
2013-01-01 00:30:00,2013-01-01 00:30:00,1081100.0,170.12000,8.460000,R80711,180.183333
2013-01-01 00:40:00,2013-01-01 00:40:00,1062010.0,171.89999,8.490000,R80711,177.001667


In [6]:
scada_df.dtypes

time              datetime64[ns]
wtur_W_avg               float64
wmet_HorWd_Dir           float64
wmet_wdspd_avg           float64
id                        object
energy_kwh               float64
dtype: object

In [7]:
scada_df.max()

time              2013-12-31 00:00:00
wtur_W_avg                2.05138e+06
wmet_HorWd_Dir                    360
wmet_wdspd_avg                  20.01
id                             R80790
energy_kwh                    341.897
dtype: object

## Step 3: Calculate Turbine Gross Energy

In [8]:
a = turbine_long_term_gross_energy.TurbineLongTermGrossEnergy(engie_project)
reanal_sub= ['ncep2', 'merra2', 'erai']

INFO:operational_analysis.methods.turbine_long_term_gross_energy:Initializing TurbineLongTermGrossEnergy Object
INFO:operational_analysis.methods.turbine_long_term_gross_energy:Processing SCADA data into dictionaries by turbine (this can take a while)


In [9]:
a.run(reanal_sub)

INFO:operational_analysis.methods.turbine_long_term_gross_energy:Filtering turbine data
INFO:operational_analysis.methods.turbine_long_term_gross_energy:Processing reanalysis data to daily averages
INFO:operational_analysis.methods.turbine_long_term_gross_energy:Processing scada data to daily sums
4it [00:00, 24.91it/s]
INFO:operational_analysis.methods.turbine_long_term_gross_energy:Setting up daily data for model fitting
INFO:operational_analysis.methods.turbine_long_term_gross_energy:Fitting model data
INFO:operational_analysis.methods.turbine_long_term_gross_energy:Applying fitting results to calculate long-term gross energy
INFO:operational_analysis.methods.turbine_long_term_gross_energy:Run completed


In [10]:
a._summary_results

,R80711,R80721,R80736,R80790
ncep2,3.40849e+06,2.7505e+06,2.91039e+06,3.01819e+06
merra2,3.3851e+06,2.74009e+06,2.90477e+06,2.99865e+06
erai,3.54745e+06,2.86515e+06,3.04267e+06,3.15675e+06


In [11]:
a._plant_gross

12242734.565417478

## Step 4: Caclulate Electrical Losses

In [12]:
e = electrical_losses.ElectricalLosses(engie_project)

INFO:operational_analysis.methods.electrical_losses:Initializing Electrical Losses Object


In [13]:
e.run()

INFO:operational_analysis.methods.electrical_losses:Processing SCADA data
INFO:operational_analysis.methods.electrical_losses:Processing meter data
INFO:operational_analysis.methods.electrical_losses:Calculating electrical losses


In [14]:
e._electrical_losses

0.01976013563371193